# OpenLattice Python API tutorial

This notebook shows how to use the open API documentation to interact with the OpenLattice backend, and load and join data for analysis with python.

After we've installed the python library for the OL-API, we can load the library, set the basepath and our authentication, and get the client. You can find you jwt token in the [**account page**](https://openlattice.com/gallery/#/home) of your openlattice account

In [1]:
import openlattice
import numpy as np
import pandas as pd

jwt = "***"

baseurl = 'https://api.openlattice.com'

configuration = openlattice.Configuration()
configuration.host = baseurl
configuration.api_key_prefix['Authorization'] = 'Bearer'
configuration.api_key['Authorization'] = jwt

In this notebook, we'll use the edmAPI (to find the details of the entity sets), the dataAPI (to load the data) and the searchAPI (to search the neighbors).

In [2]:
edmAPI = openlattice.EdmApi(openlattice.ApiClient(configuration))
dataAPI = openlattice.DataApi(openlattice.ApiClient(configuration))
searchAPI = openlattice.SearchApi(openlattice.ApiClient(configuration))

## Get original data and data from localhost

We now load the entity set IDs and the data. First, we decide what data we want.  Let's say we have integrated data from a sheriff's office, supportive housing group, and detox center. We want to look at arrestees and any housing statuses they've given in the past. Use your flight diagrams to decide upon entity sets and relationships and get their names: from your OpenLattice integration, you should have a diagram of the graph data model that your data now resides in. The one for our healthcare demonstration data is [here](https://github.com/Lattice-Works/api-clients/examples/demohealth.pdf).

We can see that **DemoPatients -> DemoLivesAt -> DemoHousings** gives us people and any housing status associated with them. Let's say you linked sheriff and health data in a new person dataset called **"DemoLinkedPeople2"**. We know that by calling upon the linked people we can include those arrested.

First, we get the entitysetid's for the datasets we want. The documentation for the edmAPI can be found here: https://github.com/Lattice-Works/api-clients/blob/master/python/docs/EdmApi.md
It lists all functions with a description. We will use the function get_entity_set_id(). If you click on that function, it shows which parameters should be added. In this case the parameters are entity_set_name, which is a string.

In [3]:
sourceEntitySetId = edmAPI.get_entity_set_id('DemoLinkedPeople2')
associationEntitySetId = edmAPI.get_entity_set_id('DemoLivesAt')
destEntitySetId = edmAPI.get_entity_set_id('DemoHousings')

With the dataAPI, we can load the data.  We can immediately read them into a pandas DataFrame.  Then we set the index to the **first** `openlattice.@id`.

In [4]:
src_data = pd.DataFrame(dataAPI.load_entity_set_data(sourceEntitySetId))
edge_data = pd.DataFrame(dataAPI.load_entity_set_data(associationEntitySetId))
dst_data = pd.DataFrame(dataAPI.load_entity_set_data(destEntitySetId))

In [5]:
# removing property multiplicity (get first entry)
for column in src_data.columns:
    src_data[column] = [x[0] if isinstance(x, list) else None for x in src_data[column]]
for column in edge_data.columns:
    edge_data[column] = [x[0] if isinstance(x, list) else None for x in edge_data[column]]
for column in dst_data.columns:
    dst_data[column] = [x[0] if isinstance(x, list) else None for x in dst_data[column]]

src_data = src_data.set_index('openlattice.@id', drop = True).dropna(how='all')
dst_data = dst_data.set_index('openlattice.@id', drop = True).dropna(how='all')
edge_data = edge_data.set_index('openlattice.@id', drop = True).dropna(how='all')

### Show heads of entity sets

In [6]:
src_data.head(5)

,nc.PersonBirthDate,nc.PersonEthnicity,nc.PersonGivenName,nc.PersonRace,nc.PersonSex,nc.PersonSurName,nc.SubjectIdentification,ol.datasource
openlattice.@id,,,,,,,,
003b0000-0000-0000-8000-0000000020d7,1955-12-28,nonhispanic,Jose,white,M,Calhoun,6dee78dc-aa81-4020-8c23-a44f6ccb7de2,Any County Sheriff
00e40000-0000-0000-8000-00000000207e,1981-01-15,nonhispanic,Jose,white,M,Kelly,b16e1e68-1919-4409-94cf-514b32596040,Anytown Police Dept.
01140000-0000-0000-8000-000000003159,1953-01-03,nonhispanic,Anna,white,F,Patton,a125db71-4299-4c02-9133-316c6b135108,Anytown Police Dept.
02a80000-0000-0000-8000-0000000020e8,1967-03-20,nonhispanic,Tamara,asian,F,Koch,ba086259-551e-4764-be3c-a3a44d34de92,Forest Outpatient Treatment
03810000-0000-0000-8000-0000000035cb,1996-08-14,hispanic,Caitlyn,white,F,Swanson,8f319acd-0366-4320-9c65-f9bb4ca12547,Anytown Police Dept.


In [7]:
dst_data.head(5)

,ol.id,ol.type
openlattice.@id,,
cef20000-0000-0000-8000-0000000032a9,INDEPENDENT LIVING,INDEPENDENT LIVING
d5000000-0000-0000-8000-000000003234,HOMELESS,HOMELESS
34490000-0000-0000-8000-0000000032ee,DEPENDENT LIVING,DEPENDENT LIVING
608b0000-0000-0000-8000-0000000031d0,RECENTLY INCARCERATED,RECENTLY INCARCERATED
74210000-0000-0000-8000-00000000325b,SOBER HOME,SOBER HOME


In [8]:
edge_data.head(5)

,general.id
openlattice.@id,
1f510000-0000-0000-8000-000000003195,852f2853-1f11-425d-9a34-a23b051a6ccf_INDEPENDE...
d7c80000-0000-0000-8000-00000000323f,4f304f4a-a022-4903-a34b-e67901076b54_HOMELESS
95d40000-0000-0000-8000-000000003216,34822c97-4a6f-4442-90bb-e647a989189f_SOBER HOME
0bb50000-0000-0000-8000-00000000326c,29c67c2e-1b11-4141-b7e2-909387096eb9_RECENTLY ...
92e50000-0000-0000-8000-00000000321d,15586b2e-08e0-424f-a60e-d365ed0650e1_INDEPENDE...


## Search the links between the entity sets with the API

### Full search (execute_entity_neighbor_search)

To join two entity sets (through an edge/association) we must find the unique ID that each data point, or entity, has in the three tables: the source, destination, and association. We call these IDs the neighbors.

The `execute_entity_neighbor_search`-function allows us to find all neighbors of an entity.  We can find the neighbors of the first entity in the data with the following arguments:

**note:** `src_data.index` is the index of the table: the entity_keys of this entity set)

In [9]:
all_neighbors = searchAPI.execute_entity_neighbor_search(sourceEntitySetId, src_data.index[0])
all_neighbors[10]

{'association_details': {'ol_id': "['6dee78dc-aa81-4020-8c23-a44f6ccb7de2_2015-75717']",
                         'openlattice_id': "['9a610000-0000-0000-8000-00000000315e']"},
 'association_entity_set': {'contacts': ['kim@openlattice.com',
                                         'jason@openlattice.com'],
                            'description': '',
                            'entity_type_id': '5bf88cde-65bc-4865-a58b-920eec4c4028',
                            'external': True,
                            'id': '6405cd23-e95f-488d-bfe1-2c937e9adca8',
                            'linked_entity_sets': None,
                            'linking': False,
                            'name': 'DemoInvolvedIn',
                            'title': 'Demo: InvolvedIn'},
 'neighbor_entity_set': {'contacts': ['kim@openlattice.com',
                                      'jason@openlattice.com'],
                         'description': 'Incidents in the OpenLattice criminal '
                   

### Filtered search (execute_filtered_entity_neighbor_search)

With the filtered search, we can be more specific about the source, destination and association. To make such a query, we need to instantiate an object of type `openapi_client.NeighborSearchFilter`.  Note that we will give the source_entity_set_id in the search function, so we leave it as an empty list here in the object.

We will only search for the first 10 entities of the source data for easier viewing.

**Always set the _other_ entity set (whether source or destination) to a blank.**

In [10]:
neighbor_filter = openlattice.NeighborSearchFilter(
    edge = [associationEntitySetId],
    src = [],
    dst = [destEntitySetId],
    entity_key_ids = src_data.index.tolist()[:10]
)

This is what the object looks like:

In [11]:
print(neighbor_filter.to_str())

{'dst': ['0fb7390e-a2b2-4855-ac46-bcce5ced9986'],
 'edge': ['668e63bc-a112-460c-8036-00fc3742ae4c'],
 'entity_key_ids': ['003b0000-0000-0000-8000-0000000020d7',
                    '00e40000-0000-0000-8000-00000000207e',
                    '01140000-0000-0000-8000-000000003159',
                    '02a80000-0000-0000-8000-0000000020e8',
                    '03810000-0000-0000-8000-0000000035cb',
                    '04f20000-0000-0000-8000-000000002076',
                    '05350000-0000-0000-8000-000000002153',
                    '05b40000-0000-0000-8000-00000000208f',
                    '05e40000-0000-0000-8000-000000002071',
                    '064c0000-0000-0000-8000-0000000031a8'],
 'src': []}


Now we can do the filtered search.

In [12]:
filtered_neighbors = searchAPI.execute_filtered_entity_neighbor_search(
    sourceEntitySetId, 
    neighbor_search_filter=neighbor_filter)

Here we have an object with as keys the source entity keys (the 10 first we gave above) and as objects the neighbors.

In [13]:
filtered_neighbors

{'02a80000-0000-0000-8000-0000000020e8': [{'associationEntitySet': {'id': '668e63bc-a112-460c-8036-00fc3742ae4c',
    'entityTypeId': '2ad709a7-da75-456f-831c-9f263ef9f510',
    'name': 'DemoLivesAt',
    'title': 'Demo: LivesAt',
    'description': '',
    'contacts': ['kim@openlattice.com', 'jason@openlattice.com'],
    'linking': False,
    'linkedEntitySets': [],
    'external': True,
    'organization': {'type': 'ORGANIZATION', 'id': 'openlatticeOrg'}},
   'associationDetails': {'general.id': ['ba086259-551e-4764-be3c-a3a44d34de92_HOMELESS'],
    'openlattice.@id': ['86c30000-0000-0000-8000-000000003208']},
   'neighborEntitySet': {'id': '0fb7390e-a2b2-4855-ac46-bcce5ced9986',
    'entityTypeId': '2fb96971-862a-4cd9-a9bc-00e89436a9d5',
    'name': 'DemoHousings',
    'title': 'Demo: Housings',
    'description': '',
    'contacts': ['kim@openlattice.com', 'jason@openlattice.com'],
    'linking': False,
    'linkedEntitySets': [],
    'external': True,
    'organization': {'type': 

## Creating the edges table

To recombine the entity sets, we first construct the edges table that holds the keys to join the tables together.

In [14]:
edges = [
    {
        "src": src_id, 
        "edge": edge['associationDetails']['openlattice.@id'][0],
        "dst": edge['neighborDetails']['openlattice.@id'][0]
    }
 for src_id, edges in filtered_neighbors.items() for edge in edges 
]

edges_table = pd.DataFrame(edges)
edges_table

,dst,edge,src
0,d5000000-0000-0000-8000-000000003234,86c30000-0000-0000-8000-000000003208,02a80000-0000-0000-8000-0000000020e8
1,cef20000-0000-0000-8000-0000000032a9,f9fd0000-0000-0000-8000-00000000321a,064c0000-0000-0000-8000-0000000031a8
2,d5000000-0000-0000-8000-000000003234,c13a0000-0000-0000-8000-0000000031c4,064c0000-0000-0000-8000-0000000031a8
3,34490000-0000-0000-8000-0000000032ee,c7340000-0000-0000-8000-000000003254,064c0000-0000-0000-8000-0000000031a8


## Combining the data with the association.

Now we can do left joins on the edges table with the other tables.

In [15]:
# get data in dataframe
src_data = pd.DataFrame(src_data)
edge_data = pd.DataFrame(edge_data)
dst_data = pd.DataFrame(dst_data)

In [16]:
recombined = edges_table.merge(
    src_data, 
    left_on='src', 
    right_index = True, 
    how="left"
).merge(
    edge_data, 
    left_on='edge', 
    right_index = True, 
    how="left"
).merge(
    dst_data, 
    left_on='dst', 
    right_index = True, 
    how="left"
)

In [17]:
recombined.head(5)

,dst,edge,src,nc.PersonBirthDate,nc.PersonEthnicity,nc.PersonGivenName,nc.PersonRace,nc.PersonSex,nc.PersonSurName,nc.SubjectIdentification,ol.datasource,general.id,ol.id,ol.type
0,d5000000-0000-0000-8000-000000003234,86c30000-0000-0000-8000-000000003208,02a80000-0000-0000-8000-0000000020e8,1967-03-20,nonhispanic,Tamara,asian,F,Koch,ba086259-551e-4764-be3c-a3a44d34de92,Forest Outpatient Treatment,ba086259-551e-4764-be3c-a3a44d34de92_HOMELESS,HOMELESS,HOMELESS
1,cef20000-0000-0000-8000-0000000032a9,f9fd0000-0000-0000-8000-00000000321a,064c0000-0000-0000-8000-0000000031a8,1994-12-12,hispanic,Anthony,other,M,Burton-Johnston,ca18962e-a0c1-401f-b171-b2435b696ff2,Forest Outpatient Treatment,ca18962e-a0c1-401f-b171-b2435b696ff2_INDEPENDE...,INDEPENDENT LIVING,INDEPENDENT LIVING
2,d5000000-0000-0000-8000-000000003234,c13a0000-0000-0000-8000-0000000031c4,064c0000-0000-0000-8000-0000000031a8,1994-12-12,hispanic,Anthony,other,M,Burton-Johnston,ca18962e-a0c1-401f-b171-b2435b696ff2,Forest Outpatient Treatment,ca18962e-a0c1-401f-b171-b2435b696ff2_HOMELESS,HOMELESS,HOMELESS
3,34490000-0000-0000-8000-0000000032ee,c7340000-0000-0000-8000-000000003254,064c0000-0000-0000-8000-0000000031a8,1994-12-12,hispanic,Anthony,other,M,Burton-Johnston,ca18962e-a0c1-401f-b171-b2435b696ff2,Forest Outpatient Treatment,ca18962e-a0c1-401f-b171-b2435b696ff2_DEPENDENT...,DEPENDENT LIVING,DEPENDENT LIVING
